<a href="https://colab.research.google.com/github/samuel0711/image-anomaly-detection/blob/main/inicio_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import io
import sys
import os
import gc
import shutil
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as skio
from skimage.transform import resize
import torch
from torch import nn #neural networks



if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir("/content/drive/My Drive/Colab Notebooks")
    sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


*   Peds1 tem 34 videos de treino e 36 de teste
*   Peds2 tem 16 videos de treino e 12 de teste (associadas a eventos anômalos)
*   Amostras de treino somente tem frames normais
*   Cada amostra de teste contém pelo menos alguns frames anômalo.
*   10 clips de teste de Peds1 e 12 de Peds2 são providenciados com 'Pixel Level Groundtruth Masks' com sufixo '_gt'


In [2]:
path = os.path.join(os.path.abspath(os.getcwd()), "UCSD_Anomaly_Dataset.v1p2/") #UCSDped1/Test/Test014/

tns = []


subfolders = [os.path.join(path, subfolder) for subfolder in os.listdir(path) if os.path.isdir(os.path.join(path, subfolder))]


#substituido arquivo que deu erro Test017/142
def extract_datasets(subfolders):
  tns = []

  for subfolder in subfolders:
    #print('subfolder: '+subfolder)
    for root, idx, filenames in os.walk(subfolder):
      #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
      if "Train" in root:
        #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
        for filename in filenames: 
            if ".tif" in filename:
              img = skio.imread(os.path.join(root, filename))/1.0
              tns.append(resize(img, (160,240), preserve_range = True))
              # print('root: ',root, filename)
              # print('\n',img.shape)
  
  np.save('train_dataset', np.array(tns, dtype = np.float))

  print(np.asarray(tns).shape)
  print(np.asarray(tns[0]).shape)

  tns = []

  for subfolder in subfolders:
    #print('subfolder: '+subfolder)
    for root, idx, filenames in os.walk(subfolder):
      #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
      if "Test" in root:
        #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
        for filename in filenames: 
            if ".tif" in filename:
              img = skio.imread(os.path.join(root, filename))/1.0
              tns.append(resize(img, (160,240), preserve_range = True))
              #print(img.shape)
  
  np.save('test_dataset', np.array(tns, dtype = np.float))

  print(np.asarray(tns).shape)
  print(np.asarray(tns[0]).shape)
  


extract_datasets(subfolders)

'''
for root, idx, filenames in os.walk(path):
  #print('root: ',root, '\nindex: ',idx,'\nfilename: ', filenames)
  for filename in filenames:
    print('root: ',root, '\nindex: ',idx,'\nfilename: ', filename)
    #print(filename)
    if '.bmp' in filename:
      img = skio.imread(os.path.join(root, filename))
      tns.append(img)
      print(img.shape)
      print(type(img))
      plt.imshow(img)
      #plt.figure()
      #print(img.shape)
'''
print(type(tns))
#print(tns)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


(9350, 160, 240)
(160, 240)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


(9209, 160, 240)
(160, 240)
<class 'list'>
